In [192]:
from bs4 import BeautifulSoup
import requests
import re
import collections
import pickle

In [193]:
# init dict
Data = collections.defaultdict(list)

In [226]:
URLs = ['https://en.wikipedia.org/wiki/List_of_animal_names',
      'https://en.wikipedia.org/wiki/List_of_fish_common_names',
      'https://en.wikipedia.org/wiki/List_of_trees_and_shrubs_by_taxonomic_family']

URL = 'https://en.wikipedia.org/wiki/List_of_fish_common_names'

page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

# Find all wikiparts
WikiRefs = soup.find_all('a')
# Create links 
WikiLinks = ['https://en.wikipedia.org/' + pages.get('href') for pages in WikiRefs 
                       if pages.get('href') != None 
                       if pages.get('href').startswith('/wiki/')]
                       # Reduces the retrieved pages (does not work)
                       #if pages.span != None 
                       #if pages.span.attrs['class'][0] == 'tocnumber']
            
# Remove duplicates            
WikiLinks = list(set(WikiLinks))

In [227]:
len(WikiLinks)

1274

In [228]:
# Init Dict
Data = collections.defaultdict(list)

# Known chapter titles
KnownDescriptionList = ['Physical characteristics', 'Description and morphology',
                        'Description', 'General description',
                        'Appearance',
                        'Characteristics']

# Lower them
KnownDescriptionListLower =[i.lower() for i in KnownDescriptionList]

# Removes references in text
ReferenceRemover = '\[\d*\]'

for WikiLink in WikiLinks:

    URL = WikiLink
    #URL = 'https://en.wikipedia.org/wiki/Crocodile'
    Response = requests.get(URL, timeout=5)
    Soup = BeautifulSoup(Response.content, "html.parser")

    # Check if the link contains description data
    Contents = [i.text.replace('[edit]', '') for i in Soup.find_all('h2')]
    if all(s not in Contents for s in KnownDescriptionList) == False:

        # Find Chapters
        for Tag in Soup.find_all('h2')[1:]:


            # Get species name
            Species = Soup.title.string\
                                .split(' - ')[0]\
                                .rstrip(' ')

            # Clean Chapter
            Chapter = Tag.text.strip().replace('[edit]', '')

            # Find all Text
            for Item in Tag.find_next_siblings('p'):
                # Check if Chapter is Description or similar
                if Chapter.lower() in KnownDescriptionListLower:
                    # Check if text belongs to current chapter
                    if Chapter in Item.find_previous_siblings('h2')[0].text.strip():
                        # Clean Text
                        CleanText = re.sub(ReferenceRemover, '', Item.text)
                        # Add to dict
                        Data[Species].append(tuple([1, CleanText]))

                else:
                    # Check if text belongs to current chapter
                    if Chapter in Item.find_previous_siblings('h2')[0].text.strip():
                        # Clean Text
                        CleanText = re.sub(ReferenceRemover, '', Item.text)
                        # Add to dict
                        Data[Species].append(tuple([0, CleanText]))



In [229]:
with open('../data/processed/dataWIKI_fish.pkl', 'wb') as f:
    pickle.dump(Data, f)

In [230]:
len(Data.keys())

374